In [5]:
#Seprate single parameter from merged netcdf file
import xarray as xr
import os

# Path to the directory containing your NetCDF files
data_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/ADO/Brenta/processed/Precipitation'

# Create separate output directories for temperature and precipitation data
prec_output_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/ADO/Brenta/Precipitation'
os.makedirs(prec_output_dir, exist_ok=True)

# Get a sorted list of NetCDF files in the directory
nc_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.nc')])

# Loop through sorted NetCDF files
for filename in nc_files:
    # Load the NetCDF file
        ds = xr.open_dataset(os.path.join(data_dir, filename))
        # Separate precipitation variables (adjust variable names as needed)
        precipitation_data = ds['tp']
        # Save precipitation data to the precipitation output directory
        prec_output_file = os.path.join(prec_output_dir, filename)
        precipitation_data.to_netcdf(prec_output_file)

        # Close the dataset
        ds.close()

In [ ]:
#Seprate multiple parameter from merged netcdf file
import xarray as xr
import os

# Path to the directory containing your NetCDF files
data_dir = '/mnt/CEPH_PROJECTS/InterTwin/Wflow/Alps_Catchment_Boundries/brenta_river'

# Create separate output directories for temperature and precipitation data
temp_output_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/Deltras/temperature'
prec_output_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/Deltras/precipitation'

os.makedirs(temp_output_dir, exist_ok=True)
os.makedirs(prec_output_dir, exist_ok=True)

# Get a sorted list of NetCDF files in the directory
nc_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.nc')])

# Loop through sorted NetCDF files
for filename in nc_files:
    # Load the NetCDF file
        ds = xr.open_dataset(os.path.join(data_dir, filename))
        # Separate temperature and precipitation variables (adjust variable names as needed)
        temperature_data = ds['temp']
        precipitation_data = ds['precip']

        # Save temperature data to the temperature output directory
        temp_output_file = os.path.join(temp_output_dir, filename)
        temperature_data.to_netcdf(temp_output_file)

        # Save precipitation data to the precipitation output directory
        prec_output_file = os.path.join(prec_output_dir, filename)
        precipitation_data.to_netcdf(prec_output_file)

        # Close the dataset
        ds.close()

In [1]:
# The line of code helpful to aggregate daily data to monthly data and then take mean for each month for all years
# and calculate mean monthly for all years
import xarray as xr
import os

# Directory containing the NetCDF files
data_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/Original_ERA5/Brenta/Precipitation/processed/Precipitation'

# List of NetCDF files
nc_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.nc')])

# Initialize an empty list to store datasets
datasets = []

# Loop through the NetCDF files and load them into a list
for filename in nc_files:
    ds = xr.open_dataset(os.path.join(data_dir, filename))
    datasets.append(ds)

# Check for consistent time coordinate and dimension
for ds in datasets:
    if 'time' not in ds.coords:
        raise ValueError(f"'time' coordinate not found in dataset: {ds}")
    if 'time' not in ds.dims:
        raise ValueError(f"'time' dimension not found in dataset: {ds}")

# Concatenate the datasets
combined_ds = xr.concat(datasets, dim='time')

# Ensure the 'time' dimension is sorted in chronological order
combined_ds = combined_ds.sortby('time')

# Resample to monthly data and compute mean
monthly_mean_ds = combined_ds.resample(time='1M').sum()

# Create an output directory for saving the results
output_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/ADO/Brenta/Precipitation/mean_monthly_precip_2000_20'
os.makedirs(output_dir, exist_ok=True)

# Loop through each month and save the corresponding data
for month in range(1, 13):  # January to December
    monthly_data = monthly_mean_ds.sel(time=monthly_mean_ds['time.month'] == month)
    
    # Compute the mean of all the years for the specific month
    monthly_data = monthly_data.mean(dim='time')
    
    output_file = os.path.join(output_dir, f'monthly_mean_precip_{month}.nc')
    monthly_data.to_netcdf(output_file)

print("Monthly mean data saved as NetCDF files.")


Monthly mean data saved as NetCDF files.


In [6]:
import xarray as xr
import os

# Directory containing the NetCDF files
data_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/ADO/Brenta/Precipitation'

# List of NetCDF files
nc_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.nc')])

# Initialize an empty list to store datasets
datasets = []

# Loop through the NetCDF files and load them into a list
for filename in nc_files:
    ds = xr.open_dataset(os.path.join(data_dir, filename))
    datasets.append(ds)

# Check for consistent time coordinate and dimension
for ds in datasets:
    if 'time' not in ds.coords:
        raise ValueError(f"'time' coordinate not found in dataset: {ds}")
    if 'time' not in ds.dims:
        raise ValueError(f"'time' dimension not found in dataset: {ds}")

# Concatenate the datasets
combined_ds = xr.concat(datasets, dim='time')

# Ensure the 'time' dimension is sorted in chronological order

combined_ds = combined_ds.sortby('time')

# Create an output directory for saving the results
output_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/ADO/Brenta/Precipitation/mean_daily_precip_2000_20'
os.makedirs(output_dir, exist_ok=True)

# Loop through each month to consider varying month lengths
for month in range(1, 13):  # January to December
    days_in_month = combined_ds.sel(time=combined_ds['time.month'] == month)['time.day'].max().values
    
    for day in range(1, days_in_month + 1):  # Iterate up to the maximum day in the month
        daily_data = combined_ds.sel(time=(combined_ds['time.month'] == month) & (combined_ds['time.day'] == day))
        
        # Compute the mean of all the years for the specific day
        daily_data = daily_data.mean(dim='time')
        
        output_file = os.path.join(output_dir, f'mean_precip_{month:02d}_{day:02d}.nc')
        daily_data.to_netcdf(output_file)

print("Mean daily data saved as NetCDF files.")


Mean daily data saved as NetCDF files.


In [14]:

import xarray as xr
import os
import pandas as pd

# Directory containing the NetCDF files with mean daily temperature data
data_dir = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/CERRA/processed/Temperature/mean_daily_temp'

# List of NetCDF files in the directory
nc_files = sorted([f for f in os.listdir(data_dir) if f.endswith('.nc')])

# Initialize an empty list to store mean results
mean_results = []

# Loop through the NetCDF files and calculate the mean for each file
for filename in nc_files:
    ds = xr.open_dataset(os.path.join(data_dir, filename))
    
    # Specify the variable for which you want to compute the mean
    temperature_variable = ds['temp']  # Replace with the actual variable name
    
    # Compute the mean temperature for the current NetCDF file
    mean_temperature = temperature_variable.mean().values.item()
    
    # Append the mean result to the list
    mean_results.append(mean_temperature)

# Create a DataFrame with the mean results
combined_df = pd.DataFrame(mean_results, columns=['Mean Temperature'])

# Save the combined DataFrame as a CSV file
csv_filename = '/mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/Deltras/temperature/mean_daily_temp/combined_mean_results.csv'  # Replace with the desired path and filename
combined_df.to_csv(csv_filename, index=False)

print(f"Combined mean results saved as CSV: {csv_filename}")


Combined mean results saved as CSV: /mnt/CEPH_PROJECTS/InterTwin/Data_Comparision/Deltras/temperature/mean_daily_temp/combined_mean_results.csv
